In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import os
import torch
import tqdm

from trade.models import set_up_sequence_INN_ScalarTheory

from trade.datasets import (
    ActionScalarTheory
    )

from trade.plots import (
    get_susceptibility,
    get_U_L,
    bootstrap
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

Load the trained Models

---

In [ ]:
path_dict = {
    "NLL":"../results/runs_ScalarTheory/Path to your trained model/lightning_logs/version_0/",
    "TRADE_grid":"../results/runs_ScalarTheory/Path to your trained modellightning_logs/version_0/",
    "reverse_KL_NLL":"../results/runs_ScalarTheory/Path to your trained model/lightning_logs/version_0/"
}

In [ ]:
def load_model_from_folder(folder,device):

    config_i = yaml.safe_load(Path(folder + "/hparams.yaml").read_text())
    state_dict_folder_i = folder + f"/checkpoints/"

    files = os.listdir(state_dict_folder_i)
    state_dict_path_i = os.path.join(state_dict_folder_i,files[0])

    config_i["device"] = device

    INN_i = set_up_sequence_INN_ScalarTheory(config=config_i)
    INN_i.load_state_dict(state_dict_path_i)
    INN_i.train(False)

    return INN_i,config_i

In [ ]:
INN_dict = {}
config_dict = {}

for key in path_dict:
    INN_key,config_key = load_model_from_folder(
        folder = path_dict[key],
        device=device
    )

    INN_dict[key] = INN_key
    config_dict[key] = config_key

Load the reference simulation

---

In [ ]:
# Load the reference simulation
reference = np.loadtxt("../data/ScalarTheory/validation_data/N_8_LANGEVIN_SPECIFIC/summary_lambda_0.02_0.txt",skiprows = 1)

Physical obervables

---

In [ ]:
n_kappas = 200
n_samples_phyiscs = 10000
bs_physics = 2000

with torch.no_grad():
    for key in INN_dict.keys():

        kappas_phyiscs = np.linspace(config_dict[key]["config_evaluation"]["kappa_physics_specs"][0],config_dict[key]["config_evaluation"]["kappa_physics_specs"][1],n_kappas)
        lambda_physics = np.arange(config_dict[key]["config_evaluation"]["lambda_physics_specs"][0],config_dict[key]["config_evaluation"]["lambda_physics_specs"][1],config_dict[key]["config_evaluation"]["lambda_physics_specs"][2])

        if len(lambda_physics) > 1:
            raise NotImplementedError
        
        output_file_i = f"./summary_phyiscs_properties_lambda_{lambda_physics[0]}_{key}.txt"
        if os.path.exists(output_file_i):
            continue

        n_batches_phyiscs = int(n_samples_phyiscs / bs_physics)

        magnetizations_i = torch.zeros([len(kappas_phyiscs),2])
        actions_gt_i = torch.zeros([len(kappas_phyiscs),2])
        susceptibility_i = torch.zeros([len(kappas_phyiscs),2])
        binder_cumulant_i = torch.zeros([len(kappas_phyiscs),2])

    
        counter = 0

        for k in tqdm.tqdm(kappas_phyiscs):
            for l in lambda_physics:

                actions_gt_kl_u = torch.zeros([0])
                magnetization_kl_u = torch.zeros([0])

                for i in range(n_batches_phyiscs):
                    with torch.no_grad():
                        samples_kli_u = INN_dict[key].sample(n_samples = bs_physics,beta_tensor = k).cpu().detach()

                        action_gt_kli_u = ActionScalarTheory(samples_kli_u,k,l)
                        magnetization_kli_u = samples_kli_u.sum(dim = (1,2,3))

                    actions_gt_kl_u = torch.cat((actions_gt_kl_u,action_gt_kli_u),0)
                    magnetization_kl_u = torch.cat((magnetization_kl_u,magnetization_kli_u),0)

                N = config_dict[key]["config_data"]["N"]

                mean_magnetization,std_magnetization = bootstrap(x = np.abs(np.array(magnetization_kl_u)) / N**2,s = np.mean,args={"axis":0})
                mean_action_gt,std_action_gt = bootstrap(x = np.array(actions_gt_kl_u) / N**2,s = np.mean,args={"axis":0})
                susceptibility_mean,sigma_susceptibility = bootstrap(x = np.abs(np.array(magnetization_kl_u)),s = get_susceptibility,args={"Omega":N**2})
                U_L_mean,sigma_U_L = bootstrap(x = np.array(magnetization_kl_u),s = get_U_L,args={"Omega":N**2})

                magnetizations_i[counter] = torch.Tensor([mean_magnetization,std_magnetization])
                actions_gt_i[counter] = torch.Tensor([mean_action_gt,std_action_gt])
                susceptibility_i[counter] = torch.Tensor([susceptibility_mean,sigma_susceptibility])
                binder_cumulant_i[counter] = torch.Tensor([U_L_mean,sigma_U_L])

                counter += 1

        #Save the results as a text file. Write the infirmation in the following order: kappa, magnetization, action, susceptibility, binder cumulant and add it to the header
        header = "kappa,mean_magnetization,std_magnetization,mean_action_gt,std_action_gt,susceptibility_mean,sigma_susceptibility,U_L_mean,sigma_U_L"
        data = np.concatenate((kappas_phyiscs[:,np.newaxis],magnetizations_i,actions_gt_i,susceptibility_i,binder_cumulant_i),axis = 1)
        np.savetxt(output_file_i,data,header = header,comments = "")


In [ ]:
colors = ["C1","C3","C6"]

labels = ["A","B","C","D"]
ylabels = [r"$\left<|m|\right>$",r"$\left<s\right>$",r"$\chi^2$",r"$U_L$"]

fs = 27
lw = 4

tags = {
    "TRADE_grid":"TRADE",
    "reverse_KL_NLL":"reverse KL + NLL",
    "NLL":"NLL"
}

reference_properties = [
    reference[:,1:3],
    reference[:,3:5],
    reference[:,7:9],
    reference[:,5:7]
]

y_lims = [
    [0.0,4.0],
    [-3,1.2],
    [-0.5,15],
    [-0.1,0.9]
]

fig_physics,ax = plt.subplots(2,2,figsize = (15,8))

#Get the properties from the saved tex files
properties_dict = {}
colors_dict = {}
c = 0

for key in INN_dict.keys():

    print(f"load data for {key}")
    data_u = np.loadtxt(f"./summary_phyiscs_properties_lambda_{lambda_physics[0]}_{key}.txt",skiprows = 1)

    magnetizations_i = torch.Tensor(data_u[:,1:3])
    actions_gt_i = torch.Tensor(data_u[:,3:5])
    susceptibility_i = torch.Tensor(data_u[:,5:7])
    binder_cumulant_i = torch.Tensor(data_u[:,7:9])

    properties_dict[key] = [data_u[:,0],magnetizations_i,actions_gt_i,susceptibility_i,binder_cumulant_i]
    colors_dict[key] = colors[c]
    c += 1

c = 1

for i in range(2):
    for j in range(2):

        ax[i][j].plot(reference[:,0],reference_properties[c-1][:,0],ls = "dotted",color = "k",label = "MCMC",lw = 1.5 * lw)

        for key in INN_dict.keys():
            
            ax[i][j].plot(properties_dict[key][0],properties_dict[key][c][:,0],ls = "-",color = colors_dict[key],label = tags[key],lw = lw)

            #Plot a marker at the position where the training data was located
            for base_param in config_dict[key]["config_data"]["init_data_set_params"]["kappa_list"]:

                #Interpolate betewwn the two closest points on the grid
                idx = np.argmin(np.abs(properties_dict[key][0] - base_param))
                ax[i][j].plot(properties_dict[key][0][idx],properties_dict[key][c][idx,0],ls = "",marker = "o",markersize = 10, color = colors_dict[key])


        ax[i][j].set_ylim(bottom = y_lims[c-1][0],top = y_lims[c-1][1])
        ax[i][j].set_xlabel(r"$\kappa$",fontsize = fs)
        ax[i][j].set_ylabel(ylabels[c-1],fontsize = fs)
        ax[i][j].set_title(labels[c-1],fontsize = fs)

        c += 1

        ax[i][j].tick_params(axis='both', which='major', labelsize=fs)

handles, labels = [], []

for handle, label in zip(*ax[0][0].get_legend_handles_labels()):
    handles.append(handle)
    labels.append(label)

# Add a single legend below all subplots
fig_physics.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=4,fontsize = fs)
plt.tight_layout()

plt.savefig("ScalarTheory_properties.pdf",bbox_inches = "tight")
plt.close()

Effective sampling size

---

In [ ]:
def get_ESS(INN,n_sampels_ESS = 10000,bs_ESS = 2000,config = None,n_kappas = 200):
    n_batches_ESS = int(n_sampels_ESS / bs_ESS)

    kappa_ESS = np.linspace(config["config_evaluation"]["kappa_physics_specs"][0],config["config_evaluation"]["kappa_physics_specs"][1],n_kappas)
    lambda_ESS = np.arange(config["config_evaluation"]["lambda_physics_specs"][0],config["config_evaluation"]["lambda_physics_specs"][1],config["config_evaluation"]["lambda_physics_specs"][2])

    if len(lambda_ESS) > 1:
        raise NotImplementedError

    ESS_r_list_i = np.zeros(len(kappa_ESS))

    for i,k in tqdm.tqdm(enumerate(kappa_ESS)):
        for l in lambda_ESS:

            log_p_target_INN = torch.zeros([0])
            log_p_theta_INN = torch.zeros([0])

            for j in range(n_batches_ESS):

                #Compute the log likelihood of the INN samples
                samples_i = INN.sample(n_samples = bs_ESS,beta_tensor = k)

                log_p_theta_INN_i = INN.log_prob(samples_i,k).cpu()
                log_p_theta_INN = torch.cat((log_p_theta_INN,log_p_theta_INN_i.detach().cpu()),0)

                log_p_target_INN_i = - ActionScalarTheory(samples_i.cpu(),kappas = k,lambdas = l).cpu()
                log_p_target_INN = torch.cat((log_p_target_INN,log_p_target_INN_i.detach().cpu()),0)

            #Compuete the relative Kish effective sample size
            log_omega = log_p_target_INN - log_p_theta_INN
            log_a = 2 * torch.logsumexp(log_omega,0)
            log_b = torch.logsumexp(2 * log_omega,0)

            ESS_r = torch.exp(log_a - log_b) / len(log_omega)
            ESS_r_list_i[i] = ESS_r

    return ESS_r_list_i,kappa_ESS

In [ ]:
for key in INN_dict.keys():

    output_file_i = f"./summary_ESS_lambda_{lambda_physics[0]}_{key}.txt"

    if os.path.exists(output_file_i):
        continue
    
    ESS_r_list_i,kappa_ESS_i = get_ESS(INN = INN_dict[key],config=config_dict[key])
            
    #Save the results as a text file. Write the infirmation in the following order: kappa, magnetization, action, susceptibility, binder cumulant and add it to the header
    header = "kappa,ESS_r"
    data = np.concatenate((kappa_ESS_i[:,np.newaxis],ESS_r_list_i[:,np.newaxis]),axis = 1)
    np.savetxt(output_file_i,data,header = header,comments = "")                      

In [ ]:
#Get the properties from the saved text files
ESS_dict = {}
kappas_dict = {}

for key in INN_dict.keys():

    data_u = np.loadtxt(f"./summary_ESS_lambda_{lambda_physics[0]}_{key}.txt",skiprows = 1)

    ESS_r_i = torch.Tensor(data_u[:,1])
    kappas_i = torch.Tensor(data_u[:,0])

    ESS_dict[key] = ESS_r_i
    kappas_dict[key] = kappas_i

fig_ESS,ax = plt.subplots(1,1,figsize = (12.5,4))

for key in INN_dict.keys():

    ax.plot(kappas_dict[key],ESS_dict[key],label = tags[key],color = colors_dict[key],lw = lw)

    #Plot a marker at the position where the training data was located
    for base_param in config_dict[key]["config_data"]["init_data_set_params"]["kappa_list"]:

        #Interpolate betewwn the two closest points on the grid
        idx = np.argmin(np.abs(kappas_dict[key] - base_param))
        ax.plot(kappas_dict[key][idx],ESS_dict[key][idx],ls = "",marker = "o",markersize = 10, color = colors_dict[key])

ax.set_xlabel(r"$\kappa$",fontsize = fs)
ax.set_ylabel("rel. ESS",fontsize = fs)
ax.tick_params(axis='both', which='major', labelsize=fs)
plt.tight_layout()

handles, labels = [], []

for handle, label in zip(*ax.get_legend_handles_labels()):
    handles.append(handle)
    labels.append(label)

# Add a single legend below all subplots
fig_ESS.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=4,fontsize = fs)
plt.tight_layout()
plt.savefig("ScalarTheory_ESS.pdf",bbox_inches = "tight")
plt.close()


Compute the ESS for different random seeds

---

In [ ]:
n_kappas = 200

experiment_folders = {
    "TRADE_grid":"../results/runs_ScalarTheory/Path to your trained model/lightning_logs/",
    "NLL":"../results/runs_ScalarTheory/Path to your trained model/lightning_logs/",
    "reverse_KL_NLL":"../results/runs_ScalarTheory/Path to your trained model/lightning_logs/"
}

for key in experiment_folders.keys():

    runs = os.listdir(experiment_folders[key])

    storage_performance_indicators = np.zeros([len(runs),3])
    storage_ESS = np.zeros([n_kappas,len(runs)+1])

    output_file_i = f"./summary_ESS_different_random_seeds_{key}.txt"

    if os.path.exists(output_file_i):
        continue

    for j,run in enumerate(runs):

        folder_ij = experiment_folders[key] + run

        INN_ij,config_ij = load_model_from_folder(folder_ij,device)

        ESS_ij,kappas_ij = get_ESS(INN = INN_ij,config=config_ij,n_kappas = n_kappas)

        storage_ESS[:,j+1] = ESS_ij

    storage_ESS[:,0] = kappas_ij

    #Save the results as text files
    np.savetxt(output_file_i,storage_ESS)

In [ ]:
fig_ESS_random_seed,ax_ESS_random_seed = plt.subplots(1,1,figsize = (25,10))

for key in experiment_folders.keys():

    data = np.loadtxt(f"./summary_ESS_different_random_seeds_{key}.txt")

    kappas = data[:,0]

    means = np.mean(data[:,1:],axis = 1)
    stds = np.std(data[:,1:],axis = 1)

    ax_ESS_random_seed.plot(kappas,means,label = tags[key],color = colors_dict[key],lw = lw)
    ax_ESS_random_seed.fill_between(kappas,means - stds,means + stds,color = colors_dict[key],alpha = 0.3)

handles, labels = [], []

for handle, label in zip(*ax_ESS_random_seed.get_legend_handles_labels()):
    handles.append(handle)
    labels.append(label)

ax_ESS_random_seed.tick_params(axis='both', which='major', labelsize=fs)
fig_ESS_random_seed.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.025), ncol=4,fontsize = fs)

ax_ESS_random_seed.set_xlabel(r"$\kappa$",fontsize = fs)
ax_ESS_random_seed.set_ylabel("rel. ESS",fontsize = fs)

plt.tight_layout()
plt.savefig("ScalarTheory_ESS_random_seeds.pdf",bbox_inches = "tight")
plt.close()